In [1]:
import csv
import os
import re

from bs4 import BeautifulSoup


path= '/Users/saml16/projects/Elections_info/static/'
constituency_path =path+"lok_sabha_2024/phase_1/phase1_constituency_list_links.csv"
dump_path = path+"lok_sabha_2024/phase_1/constituency_html_dump/"
phase_path=path+'lok_sabha_2024/phase_1/constituency_candidates_data/'

#/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/constituency_candidates_data
#/Users/saml16/projects/Elections_info/static/phase_1/constituency_candidates_data/356.csv

In [75]:
# exract candidates data from html files dump. (need the data to be downloaded)
all={}

def fetch_assets(row):
    #print(row)
    candidate_info={}
    try:
        assets_list = row.get_text().replace(u'\xa0',' ').split("~")
        #print(assets_list)
        candidate_info['quantity'] = assets_list[0].replace('Rs','').strip()
        candidate_info['words'] = assets_list[1].replace('+','').strip()
        return candidate_info
    except:
        return {'quantity':0, 'words':0}


def extract_and_save(file_name):
    html_data = ""
    with open(dump_path+file_name+".html", "r", encoding='utf-8') as file:
        html_data = file.read()

    soup = BeautifulSoup(html_data, 'html.parser')
    tables= soup.find_all('table')
    candidate_list_table=tables[3]
    rows = candidate_list_table.find_all('tr')[1:]
    candidates = []
    
    #print(candidate_list_table)

    # Extracting information from the table
    for count, row_ele in enumerate(rows):
        row = row_ele.find_all('td')
        print(row)
        candidate_info = {}
        candidate_info['serial_number'] = count
        name_link= row[0].find('a')
        candidate_info['name'] = name_link.text.strip()
        candidate_info['nem_candidate_id'] = name_link.get('href').split('=')[1]
        candidate_info['nem_link'] = name_link.get('href')
        candidate_info['party'] = row[1].get_text().strip()
        candidate_info['criminal_cases'] = row[2].get_text().strip()
        candidate_info['education'] = row[3].get_text().strip()
        candidate_info['age'] = row[4].get_text().strip()
        
        assets_list = fetch_assets(row[5])
        candidate_info['assets'] = assets_list.get('quantity')
        candidate_info['assets_in_words'] = assets_list.get('words')

        liabilities_list = fetch_assets(row[6])
        candidate_info['liabilities'] = liabilities_list.get('quantity')
        candidate_info['liabilities_in_words'] = liabilities_list.get('words')
        candidate_info['nem_constituency_id'] = file_name
        
        print(candidate_info)
        #break
        candidates.append(candidate_info)

    all[file_name]=candidates    
    fieldnames = candidates[0].keys()

    # Write the list of dictionaries to a CSV file
    with open(phase_path+file_name+'.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(candidates)
    print(len(candidates))
    print("done")



In [ ]:
# extract and save candidates data into csv file for individual constituencies.

# loop all the files in static/constituency_candidates_dump directory and call the extract_and_save function with the path and file name combines
for filename in os.listdir(dump_path):
    if filename.endswith(".html"):
        file=filename.replace('.html','')
        print(file, end=" ")
        extract_and_save(file)
    else:
        continue


In [37]:
####################### TEST extract and save candidates data ###########################
html_data = ""
with open(dump_path+"356"+".html", "r", encoding='utf-8') as file:
    html_data = file.read()

soup = BeautifulSoup(html_data, 'html.parser')
tables= soup.find_all('table')
candidate_list_table=tables[4]
####################### TEST extract and save candidates data ###########################
candidate_list_table

## Save combined data

In [79]:
# extract all candidates data from above and save it to a file. 

all_candidates=[]
for cons in all.values():
    all_candidates.extend(cons)
all_candidates

## convert respective nan to None

fieldnames = all_candidates[0].keys()
with open(path+'lok_sabha_2024/phase_1/candidates_combined'+'.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_candidates)

## Fetch candidates image based on the link in combined csv data

In [3]:
import pandas as pd
candidates_df=pd.read_csv(path+'lok_sabha_2024/phase_1/candidates_combined'+'.csv')
print(candidates_df.count())

candidates_df

serial_number           1618
name                    1618
nem_candidate_id        1618
nem_link                1618
party                   1618
criminal_cases          1618
education               1618
age                     1618
assets                  1618
assets_in_words         1618
liabilities             1618
liabilities_in_words     722
nem_constituency_id     1618
dtype: int64


,serial_number,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id
0,0,Altif,445,candidate.php?candidate_id=445,IND,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356
1,1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356
2,2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356
3,3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356
4,4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,4,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441
1614,5,Brajalal Debnath,1158,candidate.php?candidate_id=1158,IND,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441
1615,6,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,IND,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441
1616,7,Milan Pada Murasing,78,candidate.php?candidate_id=78,IND,0,10th Pass,58,"50,000",50 Thou,0,NaN,441


## Fetches data from the NEW, takes time

In [4]:
from bs4 import BeautifulSoup
import os 
import requests

candidate_list=list(candidates_df['nem_link'])

images={}
for candidate in candidate_list:
    response=requests.get('https://www.myneta.info/LokSabha2024/'+candidate)
    if response.content:
        soup = BeautifulSoup(response.content, 'html.parser')
        image_ele=soup.find_all('img', src=lambda x: x and '/images_candidate' in x)[0]
        #print(candidate, end=',\t')
        if image_ele:
            image_id=candidate[candidate.rfind("=")+1:]
            images[image_id]=image_ele
            

KeyboardInterrupt: 

In [138]:
img_res = requests.get(img_url.get('src'))
img_res
for img_id, img in images.items():
    print(img_id, img)

445 <img alt="profile image" class="w3-border w3-padding" src="https://myneta.info/images_candidate/mynetai_ews5LokSabha2024/572535fb8b001123021ee48018e85332bea6417c.jpg" width="300"/>
490 <img alt="profile image" class="w3-border w3-padding" src="https://myneta.info/images_candidate/mynetai_ews5LokSabha2024/39f38b3adaac3ab8ac55202ca7fc4dd11b2362af.jpg" width="300"/>
444 <img alt="profile image" class="w3-border w3-padding" src="https://myneta.info/images_candidate/mynetai_ews5LokSabha2024/b001968ca71568e9bf23bee8b6c146493a2fd977.jpg" width="300"/>
74 <img alt="profile image" class="w3-border w3-padding" src="https://myneta.info/images_candidate/mynetai_ews5LokSabha2024/292ff395a2df4122ac484f55d02a0959d2380f55.jpg" width="300"/>
446 <img alt="profile image" class="w3-border w3-padding" src="https://myneta.info/images_candidate/mynetai_ews5LokSabha2024/af44148100d175cc41a65baf22b8f61449999d88.jpg" width="300"/>
443 <img alt="profile image" class="w3-border w3-padding" src="https://mynet

In [141]:
# save fetched candidates images into right places.
candidate_images_folder= '/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/images/'
for img_id, img in images.items():
    image_src = img.get('src')
    if image_src:

        response = requests.get(image_src)
        if response.status_code == 200:
            image_path = os.path.join(candidate_images_folder, img_id+'.jpg')

            with open(image_path, 'wb') as img_file:
                img_file.write(response.content)
            #print(f'Saved image {image_name} to {output_folder}')
        else:
            print(f'Failed to fetch image from {image_url}')

# Testing join on the party and candidates consolidated

In [5]:
candidates_df

,serial_number,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id
0,0,Altif,445,candidate.php?candidate_id=445,IND,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356
1,1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356
2,2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356
3,3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356
4,4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,4,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441
1614,5,Brajalal Debnath,1158,candidate.php?candidate_id=1158,IND,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441
1615,6,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,IND,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441
1616,7,Milan Pada Murasing,78,candidate.php?candidate_id=78,IND,0,10th Pass,58,"50,000",50 Thou,0,NaN,441


In [8]:
candidates_clean= candidates_df.drop(columns=['serial_number'])
candidates_clean

,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id
0,Altif,445,candidate.php?candidate_id=445,IND,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356
1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356
2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356
3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356
4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356
...,...,...,...,...,...,...,...,...,...,...,...,...
1613,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441
1614,Brajalal Debnath,1158,candidate.php?candidate_id=1158,IND,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441
1615,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,IND,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441
1616,Milan Pada Murasing,78,candidate.php?candidate_id=78,IND,0,10th Pass,58,"50,000",50 Thou,0,NaN,441


In [9]:
# Define a custom function
parties_reference= {
'YSRCP':'YSR Congress Party',
 'IND': 'Independent',
 'Jammu & Kashmir National Conference':'JKNC',
 'TRS':'BRS',
 'AJSU Party':'AJSU',
 'Revolutionary Socialist Party':'RSP',
 'Kerala Congress (M)':'KC(M)',
 'Nationalist Democratic Progressive Party':'NDPP',
 'Rashtriya Loktantrik Party':'RLTP',
 'Sikkim Krantikari Morcha':'SKM'
}


#main function
#winners=pd.read_csv("/Users/saml16/projects/Elections_info/static/Loksabha2019/Winners/candidates_data.csv")
party=pd.read_csv('/Users/saml16/projects/Elections_info/static/Party/party.csv')
party=party.rename(columns={'alt':'party', 'href':'party_image_link', 'name':'party_name'})

def replace_party(party):
    return parties_reference.get(party, party)

candidates_clean['party'] = candidates_clean['party'].apply(replace_party)
combined=pd.merge(candidates_clean,party, on='party',how='left')
#combined.to_csv('/Users/saml16/projects/Elections_info/static/Loksabha2019/Winners/candidates_data_with_party.csv',index=False)
combined

,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id,party_image_link,party_name,nem_party_id
0,Altif,445,candidate.php?candidate_id=445,Independent,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356,NaN,NaN,NaN
1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356,https://myneta.info/party/lib/img/party/2.png,Bahujan Samaj Party (BSP),2.0
2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356,https://myneta.info/party/lib/img/party/1.png,Indian National Congress (INC),1.0
3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356,NaN,NaN,NaN
4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441,https://myneta.info/party/lib/img/party/3.png,Bharatiya Janata Party (BJP),3.0
1614,Brajalal Debnath,1158,candidate.php?candidate_id=1158,Independent,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441,NaN,NaN,NaN
1615,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,Independent,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441,NaN,NaN,NaN
1616,Milan Pada Murasing,78,candidate.php?candidate_id=78,Independent,0,10th Pass,58,"50,000",50 Thou,0,NaN,441,NaN,NaN,NaN


In [23]:

constituency=pd.read_csv('/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/phase1_constituency_list_links.csv')
constituency=constituency.rename(columns={'constituency_nem_id':'nem_constituency_id', 'constituency_name':'constituency'})
constituency

,state_name,constituency,nem_constituency_id
0,ANDAMAN AND NICOBAR ISLANDS,ANDAMAN AND NICOBAR ISLANDS,579
1,ARUNACHAL PRADESH,ARUNACHAL EAST,29
2,ARUNACHAL PRADESH,ARUNACHAL WEST,28
3,ASSAM,DIBRUGARH,33
4,ASSAM,JORHAT,34
...,...,...,...
97,UTTARAKHAND,NAINITAL-UDHAM SINGH NAGAR,446
98,UTTARAKHAND,TEHRI GARHWAL,443
99,WEST BENGAL,ALIPURDUARS (ST),535
100,WEST BENGAL,COOCH BEHAR (SC),534


In [24]:
combined_=pd.merge(combined,constituency, on='nem_constituency_id',how='left')
combined_

,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id,party_image_link,party_name,nem_party_id,state_name,constituency
0,Altif,445,candidate.php?candidate_id=445,Independent,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356,NaN,NaN,NaN,RAJASTHAN,JHUNJHUNU
1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356,https://myneta.info/party/lib/img/party/2.png,Bahujan Samaj Party (BSP),2.0,RAJASTHAN,JHUNJHUNU
2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356,https://myneta.info/party/lib/img/party/1.png,Indian National Congress (INC),1.0,RAJASTHAN,JHUNJHUNU
3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356,NaN,NaN,NaN,RAJASTHAN,JHUNJHUNU
4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356,NaN,NaN,NaN,RAJASTHAN,JHUNJHUNU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441,https://myneta.info/party/lib/img/party/3.png,Bharatiya Janata Party (BJP),3.0,TRIPURA,TRIPURA WEST
1614,Brajalal Debnath,1158,candidate.php?candidate_id=1158,Independent,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441,NaN,NaN,NaN,TRIPURA,TRIPURA WEST
1615,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,Independent,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441,NaN,NaN,NaN,TRIPURA,TRIPURA WEST
1616,Milan Pada Murasing,78,candidate.php?candidate_id=78,Independent,0,10th Pass,58,"50,000",50 Thou,0,NaN,441,NaN,NaN,NaN,TRIPURA,TRIPURA WEST


In [40]:
combined_['liabilities_in_words'] = combined_['liabilities_in_words'].fillna('')

In [18]:
combined_.to_csv('/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/combined_candidates_with_party1.csv', index=False)

In [19]:
import pandas as pd
combined_= pd.read_csv('/Users/saml16/projects/Elections_info/static/lok_sabha_2024/phase_1/combined_candidates_with_party1.csv')
combined_

,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id,party_image_link,party_name,nem_party_id,state_name,constituency
0,Altif,445,candidate.php?candidate_id=445,Independent,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356,,,0,RAJASTHAN,JHUNJHUNU
1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356,https://myneta.info/party/lib/img/party/2.png,Bahujan Samaj Party (BSP),2,RAJASTHAN,JHUNJHUNU
2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356,https://myneta.info/party/lib/img/party/1.png,Indian National Congress (INC),1,RAJASTHAN,JHUNJHUNU
3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356,,,0,RAJASTHAN,JHUNJHUNU
4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356,,,0,RAJASTHAN,JHUNJHUNU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441,https://myneta.info/party/lib/img/party/3.png,Bharatiya Janata Party (BJP),3,TRIPURA,TRIPURA WEST
1614,Brajalal Debnath,1158,candidate.php?candidate_id=1158,Independent,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441,,,0,TRIPURA,TRIPURA WEST
1615,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,Independent,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441,,,0,TRIPURA,TRIPURA WEST
1616,Milan Pada Murasing,78,candidate.php?candidate_id=78,Independent,0,10th Pass,58,"50,000",50 Thou,0,NaN,441,,,0,TRIPURA,TRIPURA WEST


In [17]:
combined_['nem_party_id']=combined_['nem_party_id'].fillna(0).astype(int)
combined_['party_image_link']=combined_['party_image_link'].fillna(' ')
combined_['party_name']=combined_['party_name'].fillna(' ')
combined_

,name,nem_candidate_id,nem_link,party,criminal_cases,education,age,assets,assets_in_words,liabilities,liabilities_in_words,nem_constituency_id,party_image_link,party_name,nem_party_id,state_name,constituency
0,Altif,445,candidate.php?candidate_id=445,Independent,0,Graduate Professional,34,"19,52,040",19 Lacs,"6,00,000",6 Lacs,356,,,0,RAJASTHAN,JHUNJHUNU
1,Banshidhar,490,candidate.php?candidate_id=490,BSP,0,Post Graduate,37,"10,35,000",10 Lacs,"1,30,000",1 Lacs,356,https://myneta.info/party/lib/img/party/2.png,Bahujan Samaj Party (BSP),2,RAJASTHAN,JHUNJHUNU
2,Brijendra Singh Ola,444,candidate.php?candidate_id=444,INC,0,Post Graduate,72,"16,02,84,000",16 Crore,"3,27,10,000",3 Crore,356,https://myneta.info/party/lib/img/party/1.png,Indian National Congress (INC),1,RAJASTHAN,JHUNJHUNU
3,Durga Prasad Meena,74,candidate.php?candidate_id=74,Ambedkarite Party of India,0,Graduate,62,"45,601",45 Thou,"50,000",50 Thou,356,,,0,RAJASTHAN,JHUNJHUNU
4,Hazari Lal,446,candidate.php?candidate_id=446,Bahujan Kranti Party (Marxwad-Ambedkarwad),1,8th Pass,56,"3,00,10,400",3 Crore,0,NaN,356,,,0,RAJASTHAN,JHUNJHUNU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,Biplab Kumar Deb,1156,candidate.php?candidate_id=1156,BJP,0,Graduate,53,"4,99,23,347",4 Crore,0,NaN,441,https://myneta.info/party/lib/img/party/3.png,Bharatiya Janata Party (BJP),3,TRIPURA,TRIPURA WEST
1614,Brajalal Debnath,1158,candidate.php?candidate_id=1158,Independent,0,Graduate Professional,77,"58,80,000",58 Lacs,0,NaN,441,,,0,TRIPURA,TRIPURA WEST
1615,Gouri Sankar Nandi,76,candidate.php?candidate_id=76,Independent,0,12th Pass,69,"12,20,000",12 Lacs,0,NaN,441,,,0,TRIPURA,TRIPURA WEST
1616,Milan Pada Murasing,78,candidate.php?candidate_id=78,Independent,0,10th Pass,58,"50,000",50 Thou,0,NaN,441,,,0,TRIPURA,TRIPURA WEST


In [55]:
nan_rows = 


combined_[combined_.isna().any(axis=1)]

columns_to_drop = ['column4', 'column5']
df = df.drop(columns=columns_to_drop)

KeyError: ('party', 'criminal_cases', 'age', 'liabilities', 'liabilities_in_words', 'nem_constituency_id')